In [ ]:
import os
import pickle
import time
import openai
import pandas as pd
import numpy as np
from openai import OpenAI
import concurrent.futures
from tqdm import tqdm
import random
import json

base_url = os.getenv("https://DeepSeek-R1-qdvrd.eastus2.models.ai.azure.com")
api_key=os.getenv("api_key")

print(base_url)
print(api_key)

client = OpenAI(
  base_url ="https://Meta-Llama-3-70B-Instruct-vycmo-serverless.eastus2.inference.ai.azure.com",
  api_key= "api_key",
)


None
None


In [ ]:
#####################
# llama auto annotationls
#####################

def runllama(row, max_retries=10):
    ###############################################
    #Fixed Statistics ... descriptions out of order
    ###############################################

    elig = row['a.EligibilityCriteria'].replace('\n','')
    print(elig)



    prompt = """Read the eligibility criteria from a clinical trial below and return a short summary that encapsulates the important details, including labratory values. Ensure the summary is concise and captures the essence of the clinical trial.

    Eligibility Criteria: """ + elig + """\n

    From the summary, generate any possible specific categories for the data, for example generate the fields:
    Age
    Sex
    Ethnicity
    Family Medical History
    Diagnosis History
    Medication History
    Vital Sign Values
    Laboratory Values
    Diseases

    And any additional categories you can think of
    """

    '''
    prompt = """Read the eligibility criteria from a clinical trial below and return a short summary that encapsulates the important details, including labratory values. Ensure the summary is concise and captures the essence of the clinical trial.

    Eligibility Criteria: """ + elig + """\n

    From the summary, extract the inclusion criteria and exclusion criteria formatted in JSON format, ONLY return the JSON output.

    In the JSON, for both the inclusion and exclusion criteria sections only add the following fields:
    Age: Exclude the units like "years". Example: "['> 18']"
    Sex: Standardize to the values of "male" or "female". Example: convert 'm' or 'boy' to "['male']"
    Ethnicity:
    Family Medical History:
    Diagnosis History:
    Medication History:
    Vital Sign Values: Example {{'Height': '6 ft', 'Weight': '160 lbs'}}
    Laboratory Values:
    Diseases:
    Other Statuses:

    All of these will be lists of values besides the Laboratory Values and Vital Sign Values which will be a dictionary, if there is nothing available for the section then just place an empty list

    Remember to only return the JSON output, nothing else
    """
    '''

    message_text = [{"role":"system","content":"You are a medical researcher extracting knowledge assertions from scientific literature."},
                    {"role":"user","content":prompt}]

    retry_count = 0
    base_sleep = 2  # Base sleep time in seconds
    max_tokens = 7000
    while retry_count < max_retries:
        try:
            completion = client.chat.completions.create(
                messages=message_text,
                model="azureai",
                max_tokens = max_tokens,
                temperature=0.8,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None,
                seed=42,
                extra_body={
                  "use_beam_search": True,
                  "best_of": 4
                }
            )
            out = completion.choices[0].message.content
            print(out)
            return out

        except Exception as e:
            if "429" in str(e):  # Check if the error is due to rate limiting
                wait = base_sleep * (2 ** retry_count) + random.uniform(0, 1)
                print(f"Rate limit exceeded. Waiting {wait:.2f} seconds before retrying...")
                time.sleep(wait)
            elif "400" in str(e):
                max_tokens -= 100
                print(f"Max Tokens: {max_tokens}")
            else:
                print(f"An error occurred: {e}")

            retry_count += 1

    return None

#####################
# Run auto annotations
#####################

#save_path = '/home/leadmandj/RDAS/RDAS_CTKG/eligibility_extraction/eligibility_llama3_extracted.csv'
save_path = 'topics_output.csv'
df = pd.read_csv('/content/topics.csv', index_col=False)
df = df[:10] #TEST



r,c = df.shape
llama_output = list()


for idx in range(r):
    row = df.iloc[idx]
    print(row)
    temp = runllama(row)
    print(temp)
    llama_output.append([row['m.NCTId'], row['a.EligibilityCriteria'], temp])


df = pd.DataFrame(llama_output)
df.to_csv(save_path, index=False)

https://github.com/Shailender-Youtube/DeepSeek-Azure-AI-Foundry-WebApp

In [ ]:
!pip install azure-core
!pip install azure-ai-inference
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
from azure.ai.inference.models import SystemMessage, UserMessage

load_dotenv()

AZURE_ENDPOINT = "https://DeepSeek-R1-qdvrd.eastus2.models.ai.azure.com" #os.getenv("AZURE_ENDPOINT")
AZURE_KEY ="api_key"# os.getenv("AZURE_KEY")

client = ChatCompletionsClient(
    endpoint=AZURE_ENDPOINT, #os.environ["AZURE_ENDPOINT"],
    credential=AzureKeyCredential(AZURE_KEY),#os.environ["AZURE_KEY"]),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="Can you use python for creating a frontend app?"),
    ],
)

print("Response:", response.choices[0].message.content)

Response: <think>
Okay, the user is asking if Python can be used to create a frontend app. Let me think.

First, I know Python is mainly a backend language, but there are some frameworks that let you build frontends. I should mention those. Like Tkinter for desktop GUIs. Maybe Kivy for mobile apps? Oh, and there's also frameworks that work with web frontends, like Django or Flask, but those are more backend. Wait, but for web frontends, Python isn't typically used. JavaScript is standard there. However, there are tools like Brython or Skulpt that transpile Python to JavaScript. Also, newer tools like NiceGUI or Streamlit for dashboards. Anvil is another one I've heard of for full-stack apps in Python.

I should outline the different scenarios: desktop, mobile, web. For each, list the Python frameworks. Also note that while Python can be used for frontend, it's not as common as JavaScript, especially for complex web UIs. Mention the pros and cons. Maybe give a simple example with Tkinte